---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [54]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [55]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [56]:
def answer_one():
    df1 = df.copy()
    df1 = df1.sort_values(by='Gold', ascending=False)
    return df1.index[0]
answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [57]:
def answer_two():
    df2 = df.copy()
    df2['difference'] = abs(df2['Gold'] - df2['Gold.1'])
    return df2.sort_values(by='difference', ascending=False).index[0]
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [58]:
def answer_three():
    df_3 = df.copy()
    df_3 = df_3[(df_3['Gold'] > 0) & (df_3['Gold.1'] > 0)]
    df_3['difference'] = abs(df['Gold'] - df['Gold.1']) / df['Gold.2']
    return df_3.sort_values(by = 'difference', ascending=False).index[0]
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [68]:
def answer_four():
    df4 = df.copy()
    df4['Points'] = df4['Gold.2'] * 3 + df4['Silver.2'] * 2 + df4['Bronze.2']
    return df4['Points']
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [60]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [61]:
def answer_five():
    df1 = census_df.copy()
    return df1.drop(df1[df1['SUMLEV'] == 40].index)['STNAME'].value_counts().index[0]
answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [69]:
def answer_six():
    df2 = census_df.copy()
    df2 = df2.drop(df2[df2['SUMLEV'] == 40].index)
    df2 = df2.sort_values(['STNAME', 'CENSUS2010POP'], ascending=[True, False]).groupby(['STNAME']).head(3)
    df2 = df2.groupby(['STNAME']).sum()
    return list(df2.sort_values(by='CENSUS2010POP', ascending=False).head(3).index)
answer_six()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,
Alabama,150,9,18,3,259,1406269,1406304,1407586,1410415,1414150,...,-5.210264,-4.776858,1.521180,0.782795,-4.730649,-0.997146,-0.936629,5.417202,5.195948,-0.233479
Alaska,150,12,27,6,280,478402,478402,481455,486320,492774,...,-8.076675,9.340896,-0.202602,-37.325733,-14.942448,-3.369482,20.270363,8.656139,-29.008088,-4.915660
Arizona,150,12,24,12,53,5173150,5173390,5186851,5238025,5320782,...,-10.509806,13.292912,7.464747,26.791785,28.281929,-4.086189,19.992674,14.636191,34.998776,36.508784
Arkansas,150,9,21,15,269,807152,807193,810664,822608,833594,...,18.012611,15.487208,19.531981,14.860365,18.764610,25.429853,23.248130,27.588498,23.927394,27.755652
California,150,12,27,18,169,15924150,15924274,15948261,16094942,16244137,...,-2.651916,-3.217407,-5.070710,-7.111949,-12.065453,10.408013,10.545720,9.005753,8.543736,3.920882
Colorado,150,12,24,24,77,1794424,1794276,1804943,1842353,1876037,...,27.771666,18.792262,22.893340,20.143692,28.235188,35.559626,29.992427,33.123220,31.120035,39.785916
Connecticut,150,3,3,27,13,2673320,2673353,2678448,2689918,2696308,...,-10.505871,-14.933884,-14.110408,-22.841714,-23.955999,5.019660,-0.710943,1.026736,-5.165822,-6.286792
Delaware,150,9,15,30,9,897934,897936,899791,907916,917099,...,15.045588,18.288721,17.688504,24.173863,23.526694,21.669717,26.005809,24.962252,32.310212,31.748738
District of Columbia,50,3,5,11,1,601723,601767,605126,620472,635342,...,11.560071,10.052444,9.457678,1.480094,5.601833,17.028422,15.972111,15.634120,8.378037,12.434838


### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [63]:
def answer_seven():
    df3 = census_df.copy()
    df3 = df3.drop(df3[df3['SUMLEV'] == 40].index)
    df3 = df3[['CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014',\
                'POPESTIMATE2015' ]]
    df31 = df3.max(axis=1) - df3.min(axis=1)
    return df3.loc[df31.sort_values(ascending=False).index[0]]['CTYNAME']
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [64]:
def answer_eight():
    df4 = census_df.copy()
   
    return df4[((df4['REGION'] == 1) | (df4['REGION'] == 2)) &\
               (df4['POPESTIMATE2015'] > df4['POPESTIMATE2014'] ) &\
               (df4['CTYNAME'].str.contains(r'^Washington'))][['STNAME','CTYNAME']]
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
